In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
stmnt_df=pd.read_csv('/content/drive/MyDrive/Github/spend-classifier/feb-stmnt.csv')

Mounted at /content/drive


Stripping columns names of leading and trailing whitespaces as:


---


```
map(lambda x:x.strip(),stmnt_df.columns.tolist())

```



In [45]:
stmnt_df.columns=list(map(lambda x:x.strip(),stmnt_df.columns.tolist()))

In [29]:
def filterOutNoise(regex_filter,df,column):
  filtered_out_df=df[df[column].str.contains(regex_filter,regex=True)]
  return filtered_out_df.reset_index()

The below applyFunc gets called two times, one for Debit series and second time for the Credit series.
This function is given as parameter pd.apply(here passed as input)

---

```
daily_debit_txns_df[['Debit','Credit']].apply(myApplyFunc)
```


In [30]:
def myApplyFunc(x):
  return pd.to_numeric(x.str.replace(',',''),errors='coerce')

In [49]:
regex_to_take_out_debit_txns='TO TRANSFER|DEBIT(?! SWEEP)'
daily_debit_txns_df=filterOutNoise(regex_to_take_out_debit_txns,stmnt_df,'Description')
daily_debit_txns_df[['Debit','Credit']]=daily_debit_txns_df[['Debit','Credit']].apply(myApplyFunc)
daily_debit_txns_df[daily_debit_txns_df['Debit']==daily_debit_txns_df['Debit'].max()]
total_debit=daily_debit_txns_df['Debit'].sum()
invstmnts_df=daily_debit_txns_df[daily_debit_txns_df['Description'].str.contains('DEBIT|Indian',regex=True)]
debit_df=pd.concat([invstmnts_df,daily_debit_txns_df]).drop_duplicates(ignore_index=True,keep=False)
monthly_invstment_tot=invstmnts_df['Debit'].sum()

In [75]:
invstmnts_df.columns

Index(['index', 'Txn Date', 'Value Date', 'Description', 'Ref No./Cheque No.',
       'Debit', 'Credit', 'Balance'],
      dtype='object')

In [74]:
debit_df['Shop']=debit_df['Description'].str.split('/').apply(lambda x:x[3])
spend_by_shop=debit_df.groupby('Shop')['Debit'].sum()
spend_by_shop.sort_values(ascending=False)

Shop
CREDCLUB1    26570.00
Body Mec     10000.00
JAY MALH      1021.00
PRANNAY       1000.00
SUKESH N       790.00
DATTATRA       678.00
Puneri f       645.00
RUPESH N       425.00
FINE FET       300.00
SHINDE D       295.00
bigbasket      223.99
SANDIP L       220.00
BBNOW          189.00
STELLAR        120.00
MUKLES         120.00
DIGAMBAR       100.00
MEDINET         80.00
HIFJUR A        64.00
Reliable        40.00
BAKERS P        40.00
Shubham         36.00
Bigbasket        5.00
Name: Debit, dtype: float64

In [76]:
spend_by_shop_and_date=debit_df.groupby(['Txn Date','Shop'])['Debit'].sum()
spend_by_shop_and_date.sort_values(ascending=False)

Txn Date     Shop     
17 Feb 2023  CREDCLUB1    18634.00
28 Feb 2023  Body Mec     10000.00
20 Feb 2023  CREDCLUB1     7936.00
4 Feb 2023   PRANNAY       1000.00
21 Feb 2023  DATTATRA       678.00
12 Feb 2023  FINE FET       300.00
             Puneri f       249.00
2 Feb 2023   SHINDE D       235.00
4 Feb 2023   JAY MALH       228.00
2 Feb 2023   bigbasket      223.99
28 Feb 2023  Puneri f       220.00
6 Feb 2023   JAY MALH       218.00
7 Feb 2023   BBNOW          189.00
14 Feb 2023  Puneri f       144.00
10 Feb 2023  SUKESH N       140.00
4 Feb 2023   STELLAR        120.00
17 Feb 2023  RUPESH N       115.00
10 Feb 2023  SANDIP L       110.00
26 Feb 2023  SUKESH N       110.00
3 Feb 2023   JAY MALH       108.00
19 Feb 2023  DIGAMBAR       100.00
15 Feb 2023  RUPESH N        95.00
7 Feb 2023   RUPESH N        90.00
23 Feb 2023  JAY MALH        85.00
9 Feb 2023   RUPESH N        80.00
21 Feb 2023  SUKESH N        80.00
10 Feb 2023  MUKLES          80.00
11 Feb 2023  MEDINET         80.